# Punti di interesse

In [2]:
import json
from shapely import Point, MultiPolygon, Polygon
from shapely.ops import unary_union

import geopandas as gpd
import osmnx as ox

from my_paths import *

## Stazioni BikeMi

In [ ]:
gdf = gpd.read_file(PATH_BIKEMI_RAW, sep = ';')
gdf = gdf.rename(columns = {'id_amat':'id_stazione',
                           'stalli':'n_posti',
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'location'})
gdf.drop(['stato', 'id_via', 'indirizzo', 'civico', 'zd_attuale', 'anno', "tipo"], axis=1, inplace=True)
gdf['icon'] = "share"
gdf["geometry"] = gdf.apply(lambda row: Point(row['longitudine'], row['latitudine']), axis=1)
gdf = gdf.set_geometry("geometry", crs=4326)
gdf = gdf.drop(columns="location")

c:\Matteo\Coding\Project\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver CSV does not support open option SEP
  return ogr_read(


In [ ]:
gdf.to_file(PATH_BIKEMI_CLEAN, driver="GeoJSON")

# Municipi

In [ ]:
gdf = gpd.read_file(PATH_MUNICIPI_RAW).to_crs(CRS_GRAD)
gdf["MUNICIPIO"] = gdf["MUNICIPIO"].astype(str)
gdf.to_file(PATH_MUNICIPI_CLEAN)

# Parchi

In [20]:
gdf_parchi = gpd.read_file(PATH_PARCHI_RAW)

In [ ]:
def flatten_to_multipolygon(geoms):
    flat_polygons = []
    for geom in geoms:
        if isinstance(geom, Polygon):
            flat_polygons.append(geom)
        elif isinstance(geom, MultiPolygon):
            flat_polygons.extend(list(geom.geoms))
        else:
            continue  # ignora altri tipi (es. None)
    return MultiPolygon(flat_polygons)

gdf_parchi_uniti = gpd.GeoDataFrame(
    gdf_parchi.groupby("PARCO").agg({
        'ZONA': "first",
        'AREA': "first",
        'geometry': flatten_to_multipolygon
        })
).reset_index()

gdf_parchi_uniti = gdf_parchi_uniti.set_crs(CRS_GRAD).to_crs(CRS_METR)
gdf_parchi_uniti['PERIM_M'] = gdf_parchi_uniti.geometry.apply(lambda geom: unary_union(geom).boundary.length)
gdf_parchi_uniti['AREA_MQ'] = gdf_parchi_uniti.geometry.area

gdf_parchi_uniti = gdf_parchi_uniti.to_crs(CRS_GRAD)

In [22]:
gdf_parchi_uniti.to_file(PATH_PARCHI_CLEAN, driver="GeoJSON")

# Fontane

Carichiamo Raw csv

In [ ]:
gdf = gpd.read_file(PATH_FONTANE_RAW, sep=";")
gdf["Location"] = gdf.apply(lambda row: Point(float(row["LONG_X_4326"]), float(row["LAT_Y_4326"])), axis=1)
columns = {
    'objectID':'id_fontane',
    'MUNICIPIO':'municipio',
    'NIL': "nome",
    'LONG_X_4326':'longitudine',
    'LAT_Y_4326': 'latitudine',
    'Location':'geometry'
}
gdf = gdf[columns.keys()]
gdf = gdf.rename(columns=columns)
gdf['icon'] = "events"
gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs=CRS_GRAD)
gdf.to_file(PATH_FONTANE_CLEAN, driver="GeoJSON")

# Impianti sportivi

1. Rimozione righe in cui non è segnalata una data inizio costruzione
2. Rimozione delle piste ciclabili, dato che abbiamo un dataset apposito
3. Convertiamo la data in un intero, in modo da poter filtrare agilmente su kepler

In [15]:
gdf = gpd.read_file(PATH_IMPIANTI_SPORTIVI_RAW)
gdf = gdf[["municipio", "località", "obj_id", "data_ini", "area_mq", "perim_m", "descrizione_codice", "geometry"]]
gdf = gdf.rename(columns={"data_ini": "anno_inizio", "obj_id": "id_impianto"})

gdf = gdf[gdf["anno_inizio"].notna()].reset_index()
gdf["anno_inizio"] = gdf["anno_inizio"].apply(lambda x: x.year)

gdf = gdf[~gdf["descrizione_codice"].str.contains("pista ciclabile", case=False, na=False)].reset_index()
gdf = gdf.drop(columns=["level_0", "index"], errors="ignore")

In [16]:
def group_descrizione(x):
    x = set(x)
    risposta = ""
    for riga in x:
        risposta = risposta + riga.replace("Impianti sportivi - ", "") + ", "
    return risposta.strip(", ")

def group_geometry(geos):
    new_geos = []
    for geo in geos.geometry:
        for poligon in geo.geoms:
            new_geos.append(poligon)
    return MultiPolygon(new_geos)

gdf = gdf.groupby("località").agg({
    "municipio": "first",
    "anno_inizio": "max",
    "area_mq": "sum",
    "descrizione_codice": group_descrizione,
    "geometry": group_geometry
})
gdf = gpd.GeoDataFrame(gdf)


Salviamo in Clean

In [18]:
gdf.to_file(PATH_IMPIANTI_SPORTIVI_CLEAN, driver="GeoJSON")

c:\Matteo\Coding\Project\Progetto Esame BI\.venv\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


# Biblioteche

In [19]:
gdf = gpd.read_file(PATH_BIBLIOTECHE_RAW)
print(type(gdf))
column = {
    "MUNICIPIO": "Municipio",
    "NIL": "NIL",
    "long": "Longitudine",
    "lat": "Latitudine",
    "geometry": "geometry"
}
gdf = gdf[column.keys()]
gdf["icon"] = "place"
gdf = gdf.rename(columns=column)

gdf.to_file(PATH_BIBLIOTECHE_CLEAN, driver="GeoJSON")

<class 'geopandas.geodataframe.GeoDataFrame'>


# Scuole

Carichiamo tutti i file raw come gdf

In [21]:
gdf_primarie = gpd.read_file(PATH_SCUOLE_PRIMARIE_RAW)
gdf_secondarie_1 = gpd.read_file(PATH_SCUOLE_SECONDARIE_PRIMO_RAW)
gdf_secondarie_2 = gpd.read_file(PATH_SCUOLE_SECONDARIE_SECONDO_RAW)
gdf_universita = gpd.read_file(PATH_UNIVERSITA_RAW)

Scuole primarie e secondarie di primo grado possiedono la stessa granularità quindi effettiuamo le stesse operazioni per entrambi.  
1. Filtro solo per ANNOSCOL 2023/2024  
2. Rinomino colonne che ci servono e tengo solo quelle

In [22]:
column_prim_sec1 = {
    "DENOMINAZIONE": "Denominazione",
    "GRADO": "Grado",
    "INDIRIZZO": "Indirizzo",
    "MUNICIPIO": "Municipio",
    "NIL": "Quartiere",
    "LONG_X_4326": "Longitudine",
    "LAT_Y_4326": "Latitudine",
    "geometry": "geometry"
}
# Modifico nomi e tengo solo colonne che mi serono per primarie
gdf_primarie = gdf_primarie[gdf_primarie["ANNOSCOL"] == 2324].reset_index()
gdf_primarie = gdf_primarie[column_prim_sec1.keys()]
gdf_primarie = gdf_primarie.rename(columns=column_prim_sec1)
# e secondarie 
gdf_secondarie_1 = gdf_secondarie_1[gdf_secondarie_1["ANNOSCOL"] == 2324].reset_index()
gdf_secondarie_1 = gdf_secondarie_1[column_prim_sec1.keys()]
gdf_secondarie_1 = gdf_secondarie_1.rename(columns=column_prim_sec1)
gdf_secondarie_1["Grado"] = "Scuola secondaria - Primo grado"

Nulla da sistemare, faccio solo il rename delle colonne ed estraggo dal df solo quelle

In [23]:
column_sec2_uni = {
    "DENOMINAZ": "Denominazione",
    "TIPOLOGIA": "Grado",
    "INDIRIZZO": "Indirizzo",
    "MUNICIPIO": "Municipio",
    "NIL": "NIL",
    "LONG_X_4326": "Longitudine",
    "LAT_Y_4326": "Latitudine",
    "geometry": "geometry"
}
gdf_secondarie_2 = gdf_secondarie_2[column_sec2_uni.keys()]
gdf_secondarie_2 = gdf_secondarie_2.rename(columns=column_sec2_uni)
gdf_secondarie_2["Grado"] = "Scuola secondaria - Secondo grado"

gdf_universita = gdf_universita[column_sec2_uni.keys()]
gdf_universita = gdf_universita.rename(columns=column_sec2_uni)
gdf_universita["Grado"] = "Università"

Uniamo i 4 geodataframe ripuliti e portati alla stessa granularità

In [24]:
final_gdf = gpd.pd.concat([gdf_primarie, gdf_secondarie_1, gdf_secondarie_2, gdf_universita], ignore_index=True)
final_gdf = gpd.GeoDataFrame(final_gdf, geometry=gdf_primarie.geometry.name)

Aggiungo colonna icona per la visualizzazione su kepler

In [25]:
final_gdf["icon"] = "id-card"

Salviamo il file in Clean

In [26]:
final_gdf.to_file(PATH_SCUOLE_CLEAN, driver="GeoJSON")

# Farmacie

Rinominiamo e filtriamo le colonne che ci interessano

In [27]:
gdf = gpd.read_file(PATH_FARMACIE_RAW)
column = {
    "DESCRIZIONE_FARMACIA": "Descrizione_farmacia",
    "MUNICIPIO": "Municipio",
    "NIL": "NIL",
    "LONGITUDINE": "Longitudine",
    "LATITUDINE": "Latitudine",
    "geometry": "geometry"
}
gdf = gdf[column.keys()]
gdf["icon"] = "control-on"
gdf = gdf.rename(columns=column)

Salviamo in Clean

In [28]:
gdf.to_file(PATH_FARMACIE_CLEAN, driver="GeoJSON")

# Aree Gioco

Carichiamo il dataframe e modifichiamo la "data_ini" in un intero in cui teniamo solo l'anno.

In [29]:
gdf = gpd.read_file(PATH_AREE_GIOCO_RAW)
gdf["data_ini"] = gpd.pd.to_datetime(gdf["data_ini"]).dt.year.astype("Int64")

Creo un dizionario che associa ad ogni range di obj_id la data_ini più probabile in base a quelle presenti nel df.  
Questo ci permetterà di sintetizzare le data_ini quando mancano.

In [30]:
id_ranges = list(zip(range(0, 170_001, 10_000), range(9_999, 180_001, 10_000)))

id_data = {}
for start_id, end_id in id_ranges:
    df_range = gdf[(gdf["obj_id"] >= start_id) & (gdf["obj_id"] <= end_id)]
    if not df_range.empty:
        data_counts = df_range["data_ini"].value_counts()
        id_data[gpd.pd.Interval(start_id, end_id, closed="both")] = {
            "data":  data_counts.idxmax(),
            "percentuale": round(data_counts.max() / data_counts.sum() * 100, 2)
        }

In [31]:
def get_fill_date(row, id_data_reference:dict):
    for interval, data in id_data_reference.items():
        if row["obj_id"] in interval:
            return data["data"]
    return None

# Modifico i valori mancanti di "data_ini"
valori_mancanti = gdf["data_ini"].isna() | (gdf["data_ini"].astype(str).str.strip() == "")
gdf.loc[valori_mancanti, "data_ini"] = gdf[valori_mancanti].apply(lambda row: get_fill_date(row, id_data), axis=1)

# Elimino colonne superflue
gdf = gdf.drop(columns=["id_area", "obj_id", "codice", "descrizione_codice"])

In [32]:
# Raggruppo Località e tengo la data inizio più alta
def group_geometry(geos):
    new_geos = []
    for geo in geos.geometry:
        for poligon in geo.geoms:
            new_geos.append(poligon)
    return MultiPolygon(new_geos)

gdf = gdf.groupby("località").agg({
    "municipio": "first",
    "area": "first",
    "data_ini": "max",
    "area_mq": "sum",
    "geometry": group_geometry
})
gdf = gpd.GeoDataFrame(gdf)

In [33]:
# Salvo GeoJSON pulito
gdf.to_file(PATH_AREE_GIOCO_CLEAN, driver="GeoJSON")

c:\Matteo\Coding\Project\Progetto Esame BI\.venv\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


# Teatri


- Conversione json in geojson aggiungendo geometry a partire da: glongitude e glatitude
- Filtro su "zona" != 0 perché equivale a "chiusi definitavamente"
- Filtro su "sottotipo" mantenendo solo i teatri e auditorium
- Manteniamo solo le colonne utili e rinominiamole

In [36]:
df = gpd.pd.read_json(PATH_TEATRI_RAW)
df["geometry"] = df.apply(lambda row: Point(row["glongitude"], row["glatitude"]), axis=1)
df = df[df["zona"] != 0]
df = df[df["sottotipo"].str.upper().isin(["TEATRO", "AUDITORIUM"])]
columns = {
    "denominaz": "nome_teatro",
    "sottotipo": "tipologia",
    "id": "id_teatro",
    "zona": "municipio",
    "glongitude": "longitudine",
    "glatitude": "latitudine",
    "geometry": "geometry"
}
df = df[columns.keys()]
df = df.rename(columns=columns).reset_index(drop=True)
gpd.GeoDataFrame(df, geometry="geometry").to_file(PATH_TEATRI_CLEAN, driver="GeoJSON")

c:\Matteo\Coding\Project\Progetto Esame BI\.venv\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


# Cinema

- Filtriamo solo comune Milano
- Rinominiamo e manteniamo alcune colonne
- Convertiamo in geojson creando la geometry

In [38]:
df = gpd.pd.read_json(PATH_CINEMA_RAW)

df = df[df['ccomune'] == 'Milano']
df['geometry'] = df.apply(lambda row: Point(row['clongitudine'], row['clatitudine']), axis=1)

columns = {
    "canno_inserimento": "anno_inserimento",
    "cnome": "nome",
    "clongitudine": "longitudine",
    "clatitudine": "latitudine",
    "geometry": "geometry"
}
df = df[columns.keys()]
df = df.rename(columns=columns)

gpd.GeoDataFrame(df, geometry='geometry', crs=CRS_GRAD).to_file(PATH_CINEMA_CLEAN, driver="GeoJSON")

# Musei

In [39]:
df = gpd.pd.read_json(PATH_MUSEI_RAW)

df = df[df['ccomune'] == 'Milano']
df['geometry'] = df.apply(lambda row: Point(row['clongitudine'], row['clatitudine']), axis=1)

columns = {
    "canno_inserimento": "anno_inserimento",
    "cnome": "nome",
    "clongitudine": "longitudine",
    "clatitudine": "latitudine",
    "geometry": "geometry"
}
df = df[columns.keys()]
df = df.rename(columns=columns)

gpd.GeoDataFrame(df, geometry='geometry', crs=CRS_GRAD).to_file(PATH_MUSEI_CLEAN, driver="GeoJSON")

# Ospedali

### Ingestion da openstreet maps

In [8]:
place = "Milano, Italy"

tags = {
    "amenity":  ["hospital"],
}

gdf = ox.features.features_from_place(place, tags)
gdf.to_file(PATH_OSPEDALI_RAW, driver="GeoJSON")

### Etl

In [15]:
gdf = gpd.read_file(PATH_OSPEDALI_RAW)
gdf = gdf[gdf["element"] != "relation"]
column = {
    "name": "Denominazione",
    "geometry": "geometry"
}
gdf = gdf[column.keys()]
gdf = gdf.rename(columns=column)
gdf.set_crs(CRS_GRAD).to_file(PATH_OSPEDALI_CLEAN)

# Case dell'acqua

In [4]:
# Carica il file GeoJSON
gdf = gpd.read_file(PATH_CASE_ACQUA_RAW)

# Rimuovo le colonne indesiderate
gdf = gdf.drop(columns=["CAP", "ID_NIL", "Location"])

# Rinomino le colonne
gdf = gdf.rename(columns={
    "LONG_X_4326": "longitudine",
    "LAT_Y_4326": "latitudine"
})

# Creo la colonna geometry 
gdf["geometry"] = gdf.apply(lambda row: Point(row["longitudine"], row["latitudine"]), axis=1)

# Imposto il sistema di coordinate
gdf = gdf.set_crs(CRS_GRAD)

# Esporto il GeoJSON pulito
gdf.to_file(PATH_CASE_ACQUA_CLEAN, driver="GeoJSON")
